In [15]:
%reset

In [16]:
import multiprocessing
import numpy as np
from heterogeneous_spatial_networks_funcs import filepath_str, filename_str

In [17]:
# This may or may not correspond to the number of cpus for optimal
# parallelization performance. Feel free to modify if you see fit.
cpu_num = int(np.floor(multiprocessing.cpu_count()/2))

# Initialization of identification information for this particular batch
# of spider web-inspired Delaunay-triangulated networks
date = "20240805"
batch = "A"
network = "auelp"
scheme = "rccs"

date_str = f"date_{date}"
batch_str = f"batch_{batch}"
network_str = f"network_{network}"
scheme_str = f"scheme_{scheme}"

dim_str = "dim"
b_str = "b"
xi_str = "xi"
rho_nu_str = "rho_nu"
k_str = "k"
n_str = "n"
f_n_str = "f_n"
config_str = "config"

filepath = filepath_str(network)
filename_prefix = filepath + f"{date}{batch}"

identifier_filename = filename_prefix + "-identifier" + ".txt"
dim_filename = filename_prefix + f"-{dim_str}" + ".dat"
b_filename = filename_prefix + f"-{b_str}" + ".dat"
xi_filename = filename_prefix + f"-{xi_str}" + ".dat"
rho_nu_filename = filename_prefix + f"-{rho_nu_str}" + ".dat"
k_filename = filename_prefix + f"-{k_str}" + ".dat"
n_filename = filename_prefix + f"-{n_str}" + ".dat"
f_n_filename = filename_prefix + f"-{f_n_str}" + ".dat"
config_filename = filename_prefix + f"-{config_str}" + ".dat"
params_filename = filename_prefix + "-params" + ".dat"
sample_params_filename = filename_prefix + "-sample_params" + ".dat"
sample_config_params_filename = (
    filename_prefix + "-sample_config_params" + ".dat"
)

identifier_arr = np.asarray(
    [date_str, batch_str, network_str, scheme_str, dim_str, b_str, xi_str,
     rho_nu_str, k_str, n_str, f_n_str])

# Initialization of fundamental parameters for polydisperse artificial
# end-linked polymer networks
dim_arr = np.asarray([2, 3], dtype=int)
b_arr = np.asarray([1.0])
xi_arr = np.asarray([0.98])
rho_nu_arr = np.asarray([0.85])
k_arr = np.arange(3, 9, dtype=int) # 3, 4, 5, 6, 7, 8
n_arr = np.asarray([100, 150, 224, 334, 500, 748, 1118, 1672, 2500], dtype=int) # = np.rint(np.logspace(np.log10(100), np.log10(2500), num=9)).astype(int)
f_n_arr = np.asarray([0.005, 0.01, 0.025])
config_arr = np.arange(5, dtype=int) # 0, 1, 2, 3, 4

dim_num = np.shape(dim_arr)[0]
b_num = np.shape(b_arr)[0]
xi_num = np.shape(xi_arr)[0]
rho_nu_num = np.shape(rho_nu_arr)[0]
k_num = np.shape(k_arr)[0]
n_num = np.shape(n_arr)[0]
f_n_num = np.shape(f_n_arr)[0]
config_num = np.shape(config_arr)[0]

sample_num = dim_num * b_num * xi_num * rho_nu_num * k_num * n_num * f_n_num
sample_config_num = sample_num * config_num

# Populate the network parameters array
params_arr = np.empty([sample_num, 7])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for f_n in np.nditer(f_n_arr):        
                            params_arr[sample, :] = (
                                np.asarray([dim, b, xi, rho_nu, k, n, f_n])
                            )
                            sample += 1

# Populate the network sample parameters array
sample_params_arr = np.empty([sample_num, 8])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for f_n in np.nditer(f_n_arr):
                            sample_params_arr[sample, :] = (
                                np.asarray([sample, dim, b, xi, rho_nu, k, n, f_n])
                            )
                            sample += 1

# Populate the network sample configuration parameters array
sample_config_params_arr = np.empty([sample_config_num, 9])
sample = 0
indx = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for f_n in np.nditer(f_n_arr):
                            for config in np.nditer(config_arr):
                                sample_config_params_arr[indx, :] = (
                                    np.asarray([sample, dim, b, xi, rho_nu, k, n, f_n, config])
                                )
                                indx += 1
                            sample += 1

In [18]:
from heterogeneous_spatial_networks_funcs import n_tot_arg_f_func, n_arg_n_tot_func, m_arg_stoich_func, nu_arg_m_func

nu_arr = np.empty(sample_num)

for sample in range(sample_num):
    k = params_arr[sample, 4]
    n = params_arr[sample, 5]
    f_n = params_arr[sample, 6]
    n_tot = n_tot_arg_f_func(n, f_n)
    n_nu = n_arg_n_tot_func(n_tot, n)
    m = m_arg_stoich_func(n, k)
    nu_arr[sample] = nu_arg_m_func(n_nu, m)

In [19]:
print(np.min(nu_arr))
print(np.max(nu_arr))

9.75
132.66666666666666


In [20]:
from heterogeneous_spatial_networks_funcs import n_tot_arg_f_func, n_arg_n_tot_func, L_arg_rho_func

L_arr = np.empty(sample_num)

for sample in range(sample_num):
    dim = params_arr[sample, 0]
    rho_nu = params_arr[sample, 3]
    k = params_arr[sample, 4]
    n = params_arr[sample, 5]
    f_n = params_arr[sample, 6]
    n_tot = n_tot_arg_f_func(n, f_n)
    n_nu = n_arg_n_tot_func(n_tot, n)
    L_arr[sample] = L_arg_rho_func(dim, n_nu, rho_nu)

In [21]:
b = b_arr[0]

l_nghbrhd_arr = np.empty(sample_num)

for sample in range(sample_num):
    nu = nu_arr[sample]
    r_nghbrhd = nu * b
    L = L_arr[sample]
    l_nghbrhd_arr[sample] = r_nghbrhd / L

In [22]:
print(np.min(l_nghbrhd_arr))
print(np.max(l_nghbrhd_arr))

0.028788018340969564
4.637498339524817


In [23]:
# This may or may not correspond to the number of cpus for optimal
# parallelization performance. Feel free to modify if you see fit.
cpu_num = int(np.floor(multiprocessing.cpu_count()/2))

# Initialization of identification information for this particular batch
# of spider web-inspired Delaunay-triangulated networks
date = "20240805"
batch = "B"
network = "auelp"
scheme = "rccs"

date_str = f"date_{date}"
batch_str = f"batch_{batch}"
network_str = f"network_{network}"
scheme_str = f"scheme_{scheme}"

dim_str = "dim"
b_str = "b"
xi_str = "xi"
rho_nu_str = "rho_nu"
k_str = "k"
n_str = "n"
nu_str = "nu"
config_str = "config"

filepath = filepath_str(network)
filename_prefix = filepath + f"{date}{batch}"

identifier_filename = filename_prefix + "-identifier" + ".txt"
dim_filename = filename_prefix + f"-{dim_str}" + ".dat"
b_filename = filename_prefix + f"-{b_str}" + ".dat"
xi_filename = filename_prefix + f"-{xi_str}" + ".dat"
rho_nu_filename = filename_prefix + f"-{rho_nu_str}" + ".dat"
k_filename = filename_prefix + f"-{k_str}" + ".dat"
n_filename = filename_prefix + f"-{n_str}" + ".dat"
nu_filename = filename_prefix + f"-{nu_str}" + ".dat"
config_filename = filename_prefix + f"-{config_str}" + ".dat"
params_filename = filename_prefix + "-params" + ".dat"
sample_params_filename = filename_prefix + "-sample_params" + ".dat"
sample_config_params_filename = (
    filename_prefix + "-sample_config_params" + ".dat"
)

identifier_arr = np.asarray(
    [date_str, batch_str, network_str, scheme_str, dim_str, b_str, xi_str,
     rho_nu_str, k_str, n_str, nu_str])

# Initialization of fundamental parameters for polydisperse artificial
# end-linked polymer networks
dim_arr = np.asarray([2, 3], dtype=int)
b_arr = np.asarray([1.0])
xi_arr = np.asarray([0.98])
rho_nu_arr = np.asarray([0.85])
k_arr = np.arange(3, 9, dtype=int) # 3, 4, 5, 6, 7, 8
n_arr = np.asarray([100, 150, 224, 334, 500, 748, 1118, 1672, 2500], dtype=int) # = np.rint(np.logspace(np.log10(100), np.log10(2500), num=9)).astype(int)
nu_arr = np.asarray([5, 25, 125, 525], dtype=int)
config_arr = np.arange(5, dtype=int) # 0, 1, 2, 3, 4

dim_num = np.shape(dim_arr)[0]
b_num = np.shape(b_arr)[0]
xi_num = np.shape(xi_arr)[0]
rho_nu_num = np.shape(rho_nu_arr)[0]
k_num = np.shape(k_arr)[0]
n_num = np.shape(n_arr)[0]
nu_num = np.shape(nu_arr)[0]
config_num = np.shape(config_arr)[0]

sample_num = dim_num * b_num * xi_num * rho_nu_num * k_num * n_num * nu_num
sample_config_num = sample_num * config_num

# Populate the network parameters array
params_arr = np.empty([sample_num, 7])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for nu in np.nditer(nu_arr):        
                            params_arr[sample, :] = (
                                np.asarray([dim, b, xi, rho_nu, k, n, nu])
                            )
                            sample += 1

# Populate the network sample parameters array
sample_params_arr = np.empty([sample_num, 8])
sample = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for nu in np.nditer(nu_arr):
                            sample_params_arr[sample, :] = (
                                np.asarray([sample, dim, b, xi, rho_nu, k, n, nu])
                            )
                            sample += 1

# Populate the network sample configuration parameters array
sample_config_params_arr = np.empty([sample_config_num, 9])
sample = 0
indx = 0
for dim in np.nditer(dim_arr):
    for b in np.nditer(b_arr):
        for xi in np.nditer(xi_arr):
            for rho_nu in np.nditer(rho_nu_arr):
                for k in np.nditer(k_arr):
                    for n in np.nditer(n_arr):
                        for nu in np.nditer(nu_arr):
                            for config in np.nditer(config_arr):
                                sample_config_params_arr[indx, :] = (
                                    np.asarray([sample, dim, b, xi, rho_nu, k, n, nu, config])
                                )
                                indx += 1
                            sample += 1

In [24]:
from heterogeneous_spatial_networks_funcs import m_arg_stoich_func, n_nu_arg_m_func, n_tot_arg_n_func, f_arg_n_func

f_n_arr = np.empty(sample_num)

for sample in range(sample_num):
    k = params_arr[sample, 4]
    n = params_arr[sample, 5]
    nu = params_arr[sample, 6]
    m = m_arg_stoich_func(n, k)
    n_nu = n_nu_arg_m_func(m, nu)
    n_tot = n_tot_arg_n_func(n_nu, n)
    f_n_arr[sample] = f_arg_n_func(n, n_tot)

In [25]:
print(np.min(f_n_arr))
print(np.max(f_n_arr))

0.00047596382674916705
0.11764705882352941


In [26]:
from heterogeneous_spatial_networks_funcs import m_arg_stoich_func, n_nu_arg_m_func, L_arg_rho_func

L_arr = np.empty(sample_num)

for sample in range(sample_num):
    dim = params_arr[sample, 0]
    rho_nu = params_arr[sample, 3]
    k = params_arr[sample, 4]
    n = params_arr[sample, 5]
    nu = params_arr[sample, 6]
    m = m_arg_stoich_func(n, k)
    n_nu = n_nu_arg_m_func(m, nu)
    L_arr[sample] = L_arg_rho_func(dim, n_nu, rho_nu)

In [27]:
b = b_arr[0]

l_nghbrhd_arr = np.empty(sample_num)

for sample in range(sample_num):
    nu = params_arr[sample, 6]
    r_nghbrhd = nu * b
    L = L_arr[sample]
    l_nghbrhd_arr[sample] = r_nghbrhd / L

In [28]:
print(np.min(l_nghbrhd_arr))
print(np.max(l_nghbrhd_arr))

0.0206155281280883
11.602424681415384
